In [1]:
!pip install dgl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 73.4 MB/s eta 0:00:00


In [2]:
%matplotlib inline
import os

os.environ["DGLBACKEND"] = "pytorch"
import dgl
import numpy as np
import networkx as nx
import torch
import torch.nn as nn
import dgl.function as fn
import torch.nn.functional as F
import shutil
from torch.utils.data import DataLoader
import cloudpickle
from dgl.nn import GraphConv

#### Set Path

this  segment involves directory and file operations. It creates directories, deletes directories, and extracts the contents of a ZIP file.

In [3]:
current_dir = "./"
checkpoint_path = current_dir + "save_models/model_checkpoints/" + "checkpoint"
os.makedirs(checkpoint_path, exist_ok=True)

best_model_path = current_dir + "save_models/best_model/"

folder_data_temp = current_dir +"data_temp/"
shutil.rmtree(folder_data_temp, ignore_errors=True)

path_save = current_dir + "hiv.zip"
shutil.unpack_archive(path_save, folder_data_temp)

#### Custom PyTorch Datasets

the `DGLDatasetClass` class represents a classification dataset that loads DGL graphs and associated labels, masks

, and global information from a binary file. It allows for retrieving specific items from the dataset based on index.

In [4]:
""" Classification Dataset """
class DGLDatasetClass(torch.utils.data.Dataset):
    def __init__(self, address):
            self.address=address+".bin"
            self.list_graphs, train_labels_masks_globals = dgl.load_graphs(self.address)
            num_graphs =len(self.list_graphs)
            self.labels = train_labels_masks_globals["labels"].view(num_graphs,-1)
            self.masks = train_labels_masks_globals["masks"].view(num_graphs,-1)
            self.globals = train_labels_masks_globals["globals"].view(num_graphs,-1)
    def __len__(self):
        return len(self.list_graphs)
    def __getitem__(self, idx):
        return  self.list_graphs[idx], self.labels[idx], self.masks[idx], self.globals[idx]

#### Defining Train, Validation, and Test Set

the code creates instances of the `DGLDatasetClass` class for training, validation, and test sets. The `address` parameter is set differently for each set to load the corresponding dataset. Finally, the code prints the lengths of the datasets to indicate the number of graphs in each set.

In [5]:
path_data_temp = folder_data_temp + "scaffold"+"_"+str(0)
train_set = DGLDatasetClass(address=path_data_temp+"_train")
val_set = DGLDatasetClass(address=path_data_temp+"_val")
test_set = DGLDatasetClass(address=path_data_temp+"_test")

print(len(train_set), len(val_set), len(test_set))


32901 4112 4114


#### Data Loader

the `collate` function is used to process a batch of samples by concatenating the graphs, labels, masks, and globals. The `loader` function creates dataloaders for the training, validation, and test sets using the `DataLoader` class, with the `collate` function specified to process the batches.

In [6]:
def collate(batch):
    # batch is a list of tuples (graphs, labels, masks, globals)
    # Concatenate a sequence of graphs
    graphs = [e[0] for e in batch]
    g = dgl.batch(graphs)

    # Concatenate a sequence of tensors (labels) along a new dimension
    labels = [e[1] for e in batch]
    labels = torch.stack(labels, 0)

    # Concatenate a sequence of tensors (masks) along a new dimension
    masks = [e[2] for e in batch]
    masks = torch.stack(masks, 0)

    # Concatenate a sequence of tensors (globals) along a new dimension
    globals = [e[3] for e in batch]
    globals = torch.stack(globals, 0)

    return g, labels, masks, globals


def loader(batch_size=64):
    train_dataloader = DataLoader(train_set,
                              batch_size=batch_size,
                              collate_fn=collate,
                              drop_last=False,
                              shuffle=True,
                              num_workers=1)

    val_dataloader =  DataLoader(val_set,
                             batch_size=batch_size,
                             collate_fn=collate,
                             drop_last=False,
                             shuffle=False,
                             num_workers=1)

    test_dataloader = DataLoader(test_set,
                             batch_size=batch_size,
                             collate_fn=collate,
                             drop_last=False,
                             shuffle=False,
                             num_workers=1)
    return train_dataloader, val_dataloader, test_dataloader

In [7]:
train_dataloader, val_dataloader, test_dataloader = loader(batch_size=64)

#### Defining A GNN

##### Some Variables

 the code  sets up several parameters and configurations for training a model. These include the number of tasks, the size of global features, the number of epochs, the patience for early stopping, and the model configuration dictionary specifying the sizes of different features and hidden layers.

In [8]:
#Bace dataset has 1 task. Some other datasets may have some more number of tasks, e.g., tox21 has 12 tasks.
num_tasks = 1

# Size of global feature of each graph
global_size = 200

# Number of epochs to train the model
num_epochs = 100

# Number of steps to wait if the model performance on the validation set does not improve
patience = 10

#Configurations to instantiate the model
config = {"node_feature_size":127, "edge_feature_size":12, "hidden_size":100}


 the `GNN` class represents a Graph Neural Network (GNN) model. It takes a DGL graph and global features as input and performs graph convolutions to obtain graph-level representations. The model architecture and parameters are defined in the `__init__` method, and the forward pass is implemented in the `forward` method.

In [9]:
# class GNN(nn.Module):
#     def __init__(self, config, global_size = 200, num_tasks = 1):
#         super().__init__()
#         self.config = config
#         self.num_tasks = num_tasks

#         # Node feature size
#         self.node_feature_size = self.config.get('node_feature_size', 127)

#         # Edge feature size
#         self.edge_feature_size = self.config.get('edge_feature_size', 12)

#         # Hidden size
#         self.hidden_size = self.config.get('hidden_size', 100)

#         self.conv1 = GraphConv(self.node_feature_size, self.hidden_size,allow_zero_in_degree=True)
#         self.conv2 = GraphConv(self.hidden_size, self.num_tasks,allow_zero_in_degree=True)

#     # def forward(self, g, in_feat):
#     def forward(self, mol_dgl_graph, globals):
#         mol_dgl_graph.ndata["v"]= mol_dgl_graph.ndata["v"][:,:self.node_feature_size]
#         mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:,:self.edge_feature_size]
#         h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"])
#         h = F.relu(h)
#         h = self.conv2(mol_dgl_graph, h)
#         mol_dgl_graph.ndata["h"] = h
#         return dgl.mean_nodes(mol_dgl_graph, "h")

The code snippet defines a custom `SAGEConv` module that extends the `nn.Module` class from PyTorch. It implements the GraphSAGE convolutional operation by using DGL's message passing and reduce functions. The `forward` method defines the computation flow for the graph convolution layer.

In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import dgl
from dgl.nn import GraphConv
import dgl.function as fn
from dgl.nn import SAGEConv


class SAGEConv(nn.Module):
    """Graph convolution module used by the GraphSAGE model.

    Parameters
    ----------
    in_feat : int
        Input feature size.
    out_feat : int
        Output feature size.
    aggregator_type : str
        Aggregator type, e.g., 'mean', 'max', 'sum'.
    """

    def __init__(self, in_feat, out_feat, aggregator_type='mean'):
        super(SAGEConv, self).__init__()
        self.aggregator_type = aggregator_type
        self.linear = nn.Linear(in_feat * 2, out_feat)

    def forward(self, g, h):
        with g.local_scope():
            g.ndata["h"] = h
            g.update_all(
                message_func=fn.copy_u("h", "m"),
                reduce_func=getattr(fn, self.aggregator_type)("m", "h_N"),
            )
            h_N = g.ndata["h_N"]
            h_total = torch.cat([h, h_N], dim=1)
            return self.linear(h_total)

the `GNN` class implements a graph neural network model with two layers of SAGEConv graph convolutional layers. The node and edge features of the input graph are processed, and the output is obtained by aggregating the node features and computing the mean across the graph.

In [11]:
class GNN(nn.Module):
    def __init__(self, config, global_size=200, num_tasks=1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)

        self.conv1 = SAGEConv(self.node_feature_size, self.hidden_size, aggregator_type='mean')
        self.conv2 = SAGEConv(self.hidden_size, self.num_tasks, aggregator_type='mean')

    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"] = mol_dgl_graph.ndata["v"][:, :self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:, :self.edge_feature_size]
        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"])
        h = F.relu(h)
        h = self.conv2(mol_dgl_graph, h)
        mol_dgl_graph.ndata["h"] = h
        return dgl.mean_nodes(mol_dgl_graph, "h")


#### Function to Compute Score of the Model

the `compute_score` function evaluates the model by computing the ROC AUC score for each task in a multi-task setting. It aggregates the scores and returns the average score across all tasks.

In [12]:
from sklearn.metrics import roc_auc_score

def compute_score(model, data_loader, val_size, num_tasks):
    model.eval()
    metric = roc_auc_score
    with torch.no_grad():
        prediction_all= torch.empty(0)
        labels_all= torch.empty(0)
        masks_all= torch.empty(0)
        for i, (mol_dgl_graph, labels, masks, globals) in enumerate(data_loader):
            prediction = model(mol_dgl_graph, globals)
            prediction = torch.sigmoid(prediction)
            prediction_all = torch.cat((prediction_all, prediction), 0)
            labels_all = torch.cat((labels_all, labels), 0)
            masks_all = torch.cat((masks_all, masks), 0)
        average = torch.tensor([0.])
        for i in range(num_tasks):
            a1 = prediction_all[:, i][masks_all[:,i]==1]
            a2 = labels_all[:, i][masks_all[:,i]==1]
            try:
                t = metric(a2.int().cpu(), a1.cpu()).item()
            except ValueError:
                t = 0
            average += t
    return average.item()/num_tasks

#### Loss Function

 the `loss_func` function calculates the loss for a multi-task binary classification problem using the binary cross-entropy loss (`BCEWithLogitsLoss`) with specified positive weights. It applies a mask to filter out padded or irrelevant elements, and the final loss is averaged over the valid elements.

In [13]:
def loss_func(output, label, mask, num_tasks):
    pos_weight = torch.ones((1, num_tasks))
    pos_weight
    criterion = torch.nn.BCEWithLogitsLoss(reduction='none', pos_weight=pos_weight)
    loss = mask*criterion(output,label)
    loss = loss.sum() / mask.sum()
    return loss

#### Training and Evaluation

##### Training Function

the `train_epoch` function performs one training epoch for a graph-based model. It iterates over the batches in the training data, computes the model's predictions and the training loss, performs backpropagation to compute the gradients, updates the model's parameters, and accumulates the training loss. The function returns the average training loss per iteration for the epoch.

In [14]:
def train_epoch(train_dataloader, model, optimizer):
    epoch_train_loss = 0
    iterations = 0
    model.train() # Prepare model for training
    for i, (mol_dgl_graph, labels, masks, globals) in enumerate(train_dataloader):
        prediction = model(mol_dgl_graph, globals)
        loss_train = loss_func(prediction, labels, masks, num_tasks)
        optimizer.zero_grad(set_to_none=True)
        loss_train.backward()
        optimizer.step()
        epoch_train_loss += loss_train.detach().item()
        iterations += 1
    epoch_train_loss /= iterations
    return epoch_train_loss

the train_evaluate function trains and evaluates a graph-based model for a specified number of epochs. It keeps track of the best validation score and saves the corresponding model checkpoint. After training, it copies the checkpoint files of the best model to a separate directory. Finally, it prints the final results, including the average validation score.

In [15]:
def train_evaluate():

    model = GNN(config, global_size, num_tasks)
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)

    best_val = 0
    patience_count = 1
    epoch = 1

    while epoch <= num_epochs:
        if patience_count <= patience:
            model.train()
            loss_train = train_epoch(train_dataloader, model, optimizer)
            model.eval()
            score_val = compute_score(model, val_dataloader, len(val_set), num_tasks)
            if score_val > best_val:
                best_val = score_val
                print("Save checkpoint")
                path = os.path.join(checkpoint_path, 'checkpoint.pth')
                dict_checkpoint = {"score_val": score_val}
                dict_checkpoint.update({"model_state_dict": model.state_dict(), "optimizer_state": optimizer.state_dict()})
                with open(path, "wb") as outputfile:
                    cloudpickle.dump(dict_checkpoint, outputfile)
                patience_count = 1
            else:
                print("Patience", patience_count)
                patience_count += 1

            print("Epoch: {}/{} | Training Loss: {:.3f} | Valid Score: {:.3f}".format(
            epoch, num_epochs, loss_train, score_val))

            print(" ")
            print("Epoch: {}/{} | Best Valid Score Until Now: {:.3f}".format(epoch, num_epochs, best_val), "\n")
        epoch += 1

    # best model save
    shutil.rmtree(best_model_path, ignore_errors=True)
    shutil.copytree(checkpoint_path, best_model_path)

    print("Final results:")
    print("Average Valid Score: {:.3f}".format(np.mean(best_val)), "\n")


##### Function to compute test set score of the final saved model

 the `test_evaluate` function loads the best model checkpoint, initializes a model with the same configuration, loads the model's state from the checkpoint, evaluates the model on the test dataset, and prints the test score and execution time.

In [16]:
def test_evaluate():
    final_model = GNN(config, global_size, num_tasks)
    path = os.path.join(best_model_path, 'checkpoint.pth')
    with open(path, 'rb') as f:
        checkpoint = cloudpickle.load(f)
    final_model.load_state_dict(checkpoint["model_state_dict"])
    final_model.eval()
    test_score = compute_score(final_model, test_dataloader, len(test_set), num_tasks)

    print("Test Score: {:.3f}".format(test_score), "\n")
    print("Execution time: {:.3f} seconds".format(time.time() - start_time))


##### Train the model and evaluate its performance

By calling `train_evaluate()` and `test_evaluate()` one after the other, the code performs both training and testing of the graph-based model. The `start_time` variable is used to calculate and print the total execution time for both operations.

In [18]:
import time
start_time = time.time()

train_evaluate()
test_evaluate()

Save checkpoint
Epoch: 1/100 | Training Loss: 0.261 | Valid Score: 0.529
 
Epoch: 1/100 | Best Valid Score Until Now: 0.529 

Save checkpoint
Epoch: 2/100 | Training Loss: 0.166 | Valid Score: 0.542
 
Epoch: 2/100 | Best Valid Score Until Now: 0.542 

Save checkpoint
Epoch: 3/100 | Training Loss: 0.162 | Valid Score: 0.560
 
Epoch: 3/100 | Best Valid Score Until Now: 0.560 

Save checkpoint
Epoch: 4/100 | Training Loss: 0.157 | Valid Score: 0.597
 
Epoch: 4/100 | Best Valid Score Until Now: 0.597 

Save checkpoint
Epoch: 5/100 | Training Loss: 0.152 | Valid Score: 0.652
 
Epoch: 5/100 | Best Valid Score Until Now: 0.652 

Save checkpoint
Epoch: 6/100 | Training Loss: 0.149 | Valid Score: 0.700
 
Epoch: 6/100 | Best Valid Score Until Now: 0.700 

Save checkpoint
Epoch: 7/100 | Training Loss: 0.147 | Valid Score: 0.717
 
Epoch: 7/100 | Best Valid Score Until Now: 0.717 

Save checkpoint
Epoch: 8/100 | Training Loss: 0.147 | Valid Score: 0.728
 
Epoch: 8/100 | Best Valid Score Until Now: 